In [25]:
# importing the module
import pandas as pd
import numpy as np
from os.path import join as opj
import shapely as sp
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
 
DATA_DIR = "../data/swis"  
OUT_DIR = "../data/swis/clean"
# note that when running this from jupyter lab you need to tell 
# it to back out one folder (..) -- if we were running from commandline 
# as python scrtipts/swis_merge.py, you would set DATA_DIR to "data" 
sites_excel = "Sites.xlsx"
activities_excel = "SiteActivities.xlsx"
operators_excel = "SiteOperators.xlsx"
owners_excel = "SiteOwners.xlsx"
waste_excel = "SiteWaste.xlsx"

# reading the files
sites = pd.read_excel(opj(DATA_DIR, sites_excel))    
#sites = pd.read_excel("../data/swis/Sites.xlsx")
activities = pd.read_excel(opj(DATA_DIR, activities_excel)) 
operators = pd.read_excel(opj(DATA_DIR, operators_excel))
owners = pd.read_excel(opj(DATA_DIR, owners_excel))
waste = pd.read_excel(opj(DATA_DIR, waste_excel))


In [26]:
sites
# activities.columns
sites.columns
# NOTE: the raw activities and sites excels have a SPACE in the name of the SWIS NUMBER attribute. 
# be sure to fix in excel for this script to run!

Index(['SWIS Number', 'Site Name', 'Point of Contact',
       'Site Operational Status', 'Site Regulatory Status', 'Is Archived',
       'IsClosedIllegalAbandoned', 'IsSiteInertDebrisEngineeredFill',
       'IsFinancialAssurancesResponsible', 'Absorbed On', 'Absorbed By',
       'ClosedIllegalAbandonedCategory', 'EPA Federal Registry ID', 'Latitude',
       'Longitude', 'Site ZIP Code', 'County', 'ARB District', 'SWRCB Region',
       'Local Government', 'Street Address', 'City', 'State', 'ZIP Code',
       'Reporting Agency Legal Name', 'Reporting Agency Department',
       'Enforcing Agency Legal Name', 'Enforcing Agency Department'],
      dtype='object')

In [27]:
# merging the files
    
all_df_list = [sites, activities, operators, owners, waste]

dataframe = sites
for file in [activities, operators, owners, waste]:
    dataframe = pd.merge(dataframe, file, on = "SWIS Number", how = "inner")
    
df_new=dataframe.drop_duplicates(subset='SWIS Number') 



In [28]:
# df_new
all_df_list

[     SWIS Number                                 Site Name  \
 0     01-AA-0006            Davis Street Sanitary Landfill   
 1     01-AA-0014                   Therm-Tec Of California   
 2     01-AA-0273               Marciel Ranch Landspreading   
 3     01-AC-0029  Cty of Berk Solid Waste Mgmt Center & TS   
 4     01-CR-0005                  Old West Winton Landfill   
 ...          ...                                       ...   
 5612  33-AA-0374                        MDE Compost Site 1   
 5613  37-CR-0131                      2112 Crest Drive IDS   
 5614  37-CR-0132                            Old HWY 80 IDS   
 5615  33-AA-0378       Verdant Plastics Recycling Facility   
 5616  19-AR-1275                     Easy Waste Management   
 
            Point of Contact Site Operational Status Site Regulatory Status  \
 0             Sabra Ambrose                  Closed              Permitted   
 1           Alyssa Williams                 Planned               Proposed   
 2   

In [29]:
#include only "Active" and "Planned" Sites

activesites = ["Active", "Planned"]
usefulsites = df_new[df_new['Site Operational Status'].isin(activesites)== True]

usefulsites

,SWIS Number,Site Name_x,Point of Contact_x,Site Operational Status_x,Site Regulatory Status_x,Is Archived_x,IsClosedIllegalAbandoned_x,IsSiteInertDebrisEngineeredFill_x,IsFinancialAssurancesResponsible_x,Absorbed On_x,...,Site Regulatory Status,Site Type,Latitude,Longitude,Point of Contact,Activity Is Archived_y,Activity Operational Status,Activity Regulatory Status,Activity Category,Activity Classification_y
0,01-AC-0029,Cty of Berk Solid Waste Mgmt Center & TS,Lily Tieu,Active,Permitted,No,Yes,No,No,NaN,...,Permitted,Non-Disposal Only,37.88034,-122.30579,Lily Tieu,No,Active,Permitted,Transfer/Processing,Solid Waste Facility
3,04-AA-0017,North Valley Disposal Transfer Station,Eric Kiruja,Active,Permitted,No,No,No,No,NaN,...,Permitted,Non-Disposal Only,39.71274,-121.80926,Eric Kiruja,No,Active,Permitted,Transfer/Processing,Solid Waste Facility
14,07-AA-0044,WCCSLF Organic Materials Processing,Harprit Mattu,Active,Permitted,No,No,No,No,NaN,...,Permitted,Non-Disposal Only,37.96971,-122.37896,Harprit Mattu,No,Active,Permitted,Composting,Solid Waste Facility
28,08-AA-0001,Gasquet Transfer Station,Eric Kiruja,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,41.84750,-123.95806,Eric Kiruja,No,Active,Notification,Transfer/Processing,Solid Waste Operation
29,09-AA-0002,"South Tahoe Refuse Co.,Inc., T.S/MRF",John Loane,Active,Permitted,No,No,No,No,NaN,...,Permitted,Non-Disposal Only,38.91790,-120.00202,John Loane,Yes,Active,Permitted,Transfer/Processing,Solid Waste Facility
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6157,45-AA-0064,Eddie Axner Recycle Yard,Kelsey Orr,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,40.50715,-122.39646,Kelsey Orr,No,Active,Notification,Composting,Solid Waste Facility
6158,47-AA-0059,UC Davis Cooperation Extension Siskiyou,Margaret Comotto,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,41.57324,-121.28222,Margaret Comotto,No,Active,Notification,Composting,Solid Waste Facility
6159,33-AA-0374,MDE Compost Site 1,Theodore Tasiopoulos,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,33.55479,-116.16497,Theodore Tasiopoulos,No,Active,Notification,Composting,Solid Waste Facility
6160,33-AA-0378,Verdant Plastics Recycling Facility,Theodore Tasiopoulos,Planned,Proposed,No,No,No,No,NaN,...,Proposed,Non-Disposal Only,34.01206,-117.50951,Theodore Tasiopoulos,No,Planned,Proposed,Transfer/Processing,Solid Waste Facility


In [30]:
#Include only compost and transfer stations sites
print(df_new.columns.tolist())

['SWIS Number', 'Site Name_x', 'Point of Contact_x', 'Site Operational Status_x', 'Site Regulatory Status_x', 'Is Archived_x', 'IsClosedIllegalAbandoned_x', 'IsSiteInertDebrisEngineeredFill_x', 'IsFinancialAssurancesResponsible_x', 'Absorbed On_x', 'Absorbed By_x', 'ClosedIllegalAbandonedCategory_x', 'EPA Federal Registry ID_x', 'Latitude_x', 'Longitude_x', 'Site ZIP Code_x', 'County_x', 'ARB District_x', 'SWRCB Region_x', 'Local Government_x', 'Street Address_x', 'City_x', 'State_x', 'ZIP Code_x', 'Reporting Agency Legal Name_x', 'Reporting Agency Department_x', 'Enforcing Agency Legal Name_x', 'Enforcing Agency Department_x', 'Site Name_y', 'Activity_x', 'Activity Is Archived_x', 'OperationalStatus', 'RegulatoryStatus', 'Category', 'Activity Classification_x', 'WDRNo', 'WDR Landfill Class', 'Cease Operation', 'Cease Operation Type', 'Inspection Frequency', 'Throughput', 'ThroughputUnits', 'RemainingCapacity', 'Remaining Capacity Date', 'Capacity', 'CapacityUnits', 'Total Acreage', 'D

In [31]:
#include only "Composting" as 'Category' 
#do we want to add transfer stations?

activity = ["Composting"]
composters = usefulsites[usefulsites['Category'].isin(activity)== True]

composters

,SWIS Number,Site Name_x,Point of Contact_x,Site Operational Status_x,Site Regulatory Status_x,Is Archived_x,IsClosedIllegalAbandoned_x,IsSiteInertDebrisEngineeredFill_x,IsFinancialAssurancesResponsible_x,Absorbed On_x,...,Site Regulatory Status,Site Type,Latitude,Longitude,Point of Contact,Activity Is Archived_y,Activity Operational Status,Activity Regulatory Status,Activity Category,Activity Classification_y
140,27-AA-0010,Monterey Peninsula Landfill,Eric Kiruja,Active,Permitted,No,No,No,Yes,NaN,...,Permitted,Both Disposal and Non-Disposal,36.70961,-121.76223,Eric Kiruja,No,Active,Permitted,Composting,Solid Waste Facility
184,28-AA-0026,Upper Valley Disposal Service,Kate Whitney,Active,Permitted,No,No,No,No,NaN,...,Permitted,Non-Disposal Only,38.47512,-122.43835,Kate Whitney,Yes,Active,Permitted,Composting,Solid Waste Operation
194,30-AB-0378,Golden Rain Foundation Composting Op.,Megan Emslander,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,33.60377,-117.72490,Megan Emslander,No,Active,Notification,Composting,Solid Waste Operation
329,43-AN-0017,Newby Island Compost Facility,Eric Kiruja,Active,Permitted,No,No,No,No,NaN,...,Permitted,Non-Disposal Only,37.45470,-121.92300,Eric Kiruja,No,Active,Permitted,Composting,Solid Waste Facility
365,50-AA-0016,Gilton Resource Recovery Composting Fac.,Justin Vlach,Active,Permitted,No,No,No,No,NaN,...,Permitted,Non-Disposal Only,37.62099,-120.92583,Justin Vlach,No,Active,Permitted,Composting,Solid Waste Facility
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149,38-AA-0027,Golden Gate Park,Harprit Mattu,Active,Excluded,No,No,No,No,NaN,...,Excluded,Non-Disposal Only,37.77699,-122.48079,Harprit Mattu,No,Active,Excluded,Composting,Solid Waste Operation
6153,49-AA-0439,Annapolis Chip and Grind Operation,Alyssa Williams,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,38.70574,-123.34076,Alyssa Williams,No,Active,Notification,Composting,Solid Waste Facility
6157,45-AA-0064,Eddie Axner Recycle Yard,Kelsey Orr,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,40.50715,-122.39646,Kelsey Orr,No,Active,Notification,Composting,Solid Waste Facility
6158,47-AA-0059,UC Davis Cooperation Extension Siskiyou,Margaret Comotto,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,41.57324,-121.28222,Margaret Comotto,No,Active,Notification,Composting,Solid Waste Facility


In [32]:
#new composters file with only desired categories
selected_columns = composters[["SWIS Number","Site Name","Site Operational Status", "Site Regulatory Status", "Latitude", "Longitude", 'Site ZIP Code_x', 'County_x', 'ARB District_x', 'SWRCB Region_x', "Throughput", "ThroughputUnits", "RemainingCapacity", "Capacity", "CapacityUnits", "Total Acreage", "DisposalAcreage", "Waste Type"]] 


composters_new = selected_columns.copy()

composters_new.rename(columns = {'SWIS Number':'SWISNo'}, inplace = True)
print(composters_new) 



          SWISNo                                 Site Name  \
140   27-AA-0010               Monterey Peninsula Landfill   
184   28-AA-0026             Upper Valley Disposal Service   
194   30-AB-0378     Golden Rain Foundation Composting Op.   
329   43-AN-0017             Newby Island Compost Facility   
365   50-AA-0016  Gilton Resource Recovery Composting Fac.   
...          ...                                       ...   
6149  38-AA-0027                          Golden Gate Park   
6153  49-AA-0439        Annapolis Chip and Grind Operation   
6157  45-AA-0064                  Eddie Axner Recycle Yard   
6158  47-AA-0059   UC Davis Cooperation Extension Siskiyou   
6159  33-AA-0374                        MDE Compost Site 1   

     Site Operational Status Site Regulatory Status  Latitude  Longitude  \
140                   Active              Permitted  36.70961 -121.76223   
184                   Active              Permitted  38.47512 -122.43835   
194                   Activ

In [33]:
#determine all existing units for capacity
composters_new['CapacityUnits'].value_counts(dropna=False)

Tons per year           118
Cubic Yards per year    100
Cubic Yards              62
NaN                      32
Tons                     11
Tons per day             11
Cubic Yards per Day       1
Tires per day             1
Name: CapacityUnits, dtype: int64

In [34]:
composters_new[composters_new.CapacityUnits=="Tons"]

,SWISNo,Site Name,Site Operational Status,Site Regulatory Status,Latitude,Longitude,Site ZIP Code_x,County_x,County_x,ARB District_x,SWRCB Region_x,Throughput,ThroughputUnits,RemainingCapacity,Capacity,CapacityUnits,Total Acreage,DisposalAcreage,Waste Type
1370,13-AA-0095,Imperial Valley Compost @ Mesquite Cattl,Active,Notification,32.97484,-115.40676,92227,Imperial,Imperial,Imperial,Colorado River,44000.0,Tons per year,NaN,32000.0,Tons,25.0,NaN,Agricultural
1883,34-AA-0191,"Waste Management Recycle America, LLC",Active,Permitted,38.54306,-121.41306,95826,Sacramento,Sacramento,Sacramento Metro,Central Valley,1000.0,Tons per day,NaN,1500.0,Tons,12.0,NaN,Green Materials
2927,13-AA-0093,Brandt Company (Composting),Active,Notification,33.16811,-115.56598,92233,Imperial,Imperial,Imperial,Colorado River,1000.0,Cubic Yards per Day,NaN,40000.0,Tons,NaN,NaN,Agricultural
2984,13-AA-0108,Imperial Valley Compost @ Foster Feed Ya,Active,Notification,32.90708,-115.42980,92227,Imperial,Imperial,Imperial,Colorado River,45000.0,Tons per year,NaN,30000.0,Tons,25.0,NaN,Agricultural
3972,15-AA-0311,Mt Vernon Ave Recycling & Composting Fac,Active,Permitted,35.32753,-118.96750,93307,Kern,Kern,San Joaquin Valley Unified,Central Valley,976.0,Tons per day,NaN,84783.0,Tons,82.0,NaN,Food Wastes
4498,08-AA-0023,"Hambro Forest Products, Inc.",Active,Notification,41.75210,-124.17450,95531,Del Norte,Del Norte,North Coast Unified,North Coast,260.0,Tons per month,NaN,1574.0,Tons,NaN,NaN,Green Materials
4500,57-AA-0032,Recology Davis Green Material TransferOp,Active,Notification,38.55429,-121.66203,95616,Yolo,Yolo,Yolo-Solano,Central Valley,150.0,Tons per day,NaN,12000.0,Tons,15.0,NaN,Green Materials
4876,27-AA-0086,Guziks Good Humus,Active,Notification,36.52214,-121.41114,93926,Monterey,Monterey,Monterey Bay Unified,Central Coast,15000.0,Tons per year,NaN,20000.0,Tons,52.0,NaN,Agricultural
5090,27-AA-0095,Eade Ranch,Active,Notification,36.12732,-121.01021,93954,Monterey,Monterey,Monterey Bay Unified,Central Coast,5000.0,Tons,NaN,12500.0,Tons,40.0,NaN,Agricultural
5307,49-AA-0422,Airport Landfill Chip & Grind Operation,Active,Notification,38.50349,-122.82593,95492,Sonoma,Sonoma,Bay Area,North Coast,199.0,Tons per day,NaN,199.0,Tons,NaN,NaN,Green Materials


In [35]:
SWISNosToChangeToTPY = ["13-AA-0095", "13-AA-0093", "13-AA-0108", "08-AA-0023", "57-AA-0032", "27-AA-0086", "27-AA-0095"]
SWISNosToChangeToTPD = ["34-AA-0191", "49-AA-0422"]
SWISNosToChangeToTPM = ["07-AA-0070"]

#rename 'Tons' to "Tons/year"
for index, row in composters_new.iterrows():
#     if row.CapacityUnits == "Tons":
#         composters_new.at[index, 'CapacityUnits'] = "Tons/year"
        
    # becomes
    if row.SWISNo in SWISNosToChangeToTPY:
        composters_new.at[index, 'CapacityUnits'] = "Tons per year"
    if row.SWISNo in SWISNosToChangeToTPD:
        composters_new.at[index, 'CapacityUnits'] = "Tons per day"
    if row.SWISNo in SWISNosToChangeToTPM:
        composters_new.at[index, 'CapacityUnits'] = "Tons per month"
        
    
#rename 'Cubic Yards' to "Cubic yards/"
composters_new[composters_new.CapacityUnits=="Cubic Yards"]
SWISNosToChangeToCYPY = ["43-AN-0017", "36-AA-0445", "21-AA-0068", "30-AB-0468", "33-AA-0371", "15-AA-0307", "24-AA-0019", "48-AA-0083", "56-AA-0130", "40-AA-0037", "01-AA-0325", "15-AA-0287", "44-AA-0004", "50-AA-0020", "12-AA-0113", "56-AA-0142", "36-AA-0479", "42-AA-0087", "33-AA-0376", "42-AA-0053", "01-AA-0289", "28-AA-0037", "19-AR-1222", "19-AR-1226", "37-AA-0984", "37-AA-0992", "36-AA-0503", "16-AA-0022", "19-AA-0867", "33-AA-0292", "37-AA-0946", "50-AA-0018", "33-AA-0297", "17-AA-0014", "13-AA-0121", "23-AA-0029", "43-AA-0015", "57-AA-0029", "37-AA-0949", "34-AA-0219", "48-AA-0088", "39-AA-0051", "33-AA-0357", "10-AA-0182", "37-AA-0907", "37-AB-0005", "16-AA-0026", "37-AA-0957", "11-AA-0039", "30-AB-0469"]
SWISNosToChangeToCYPM = ["24-AA-0029", "24-AA-0020", "44-AA-0013"]
SWISNosToChangeToCYPD = ["19-AA-0855", "39-AA-0024", "30-AB-0474", "19-AA-0050", "04-AA-0031", "44-AA-0009", "24-AA-0039"]

for index, row in composters_new.iterrows():

    if row.SWISNo in SWISNosToChangeToCYPY:
        composters_new.at[index, 'CapacityUnits'] = "Cubic Yards per year"
    if row.SWISNo in SWISNosToChangeToCYPM:
        composters_new.at[index, 'CapacityUnits'] = "Cubic Yards per month"
    if row.SWISNo in SWISNosToChangeToCYPD:
        composters_new.at[index, 'CapacityUnits'] = "Cubic Yards per Day"
        
#rename 'NaN' to appropriate units - not able to do this

#composters_new[composters_new.CapacityUnits== "NaN"]

        
composters_new['CapacityUnits'].value_counts(dropna=False) 


Cubic Yards per year     149
Tons per year            125
NaN                       32
Tons per day              13
Cubic Yards per Day        8
Cubic Yards                3
Cubic Yards per month      3
Tons                       1
Tires per day              1
Tons per month             1
Name: CapacityUnits, dtype: int64

In [36]:
#make new capacity column with tons/year units

# first filter out all the null values
composters_new = composters_new[composters_new['Capacity'].notnull()]

#composters_new.reset_index(inplace=True)

composters_new['cap_tpy'] = 0.0


# write function to convert all capacity units into tons/year!

#how to assign values:
# df.at[i, 'CapacityUnits'] = "Tons/year"

print("CLEANING CAPACITY UNITS- CONVERT TO TONS / YEAR")
#n = len(composters_new.index)
#for i in range(n):
    
for index, row in composters_new.iterrows():
    #print("index: ", i)
    if row.CapacityUnits == "Tons per year":
        #print("tons/year")
        # tons/year * cu yards/ton * cu meters/cu yards  
        #df.cap_m3[i] = df.Capacity[i] * 2.24 * 0.764555 
        composters_new.at[index, 'cap_tpy'] = row.Capacity
        #print(composters_new.cap_tpy[i])
    elif row.CapacityUnits == "Cubic Yards per year":
        # print("cu yrds/year")
        # cu yards/year * cu meters/cu yards 
        composters_new.at[index, 'cap_tpy'] = row.Capacity * (1/2.24) 
        # print(df.cap_m3[i])
    elif row.CapacityUnits == "Cubic Yards per Day":
        composters_new.at[index, 'cap_tpy'] = row.Capacity * (1/2.24) * (365/1) 
        # print(df.cap_m3[i])
    elif row.CapacityUnits == "Cubic Yards per month":
        composters_new.at[index, 'cap_tpy'] = row.Capacity * (1/2.24) * (12/1)
    elif row.CapacityUnits == "Tons per day":
        composters_new.at[index, 'cap_tpy'] = row.Capacity * (365/1) 
    elif row.CapacityUnits == "Tons per month":
        composters_new.at[index, 'cap_tpy'] = row.Capacity * (1/2.24) * 12
        # print("tons/month")
        # print(composters_new.cap_tpy[i])
    elif row.CapacityUnits == "Tires per day":
        composters_new.at[index, 'cap_tpy'] = 0.0
    else:
        print("none of the above")


CLEANING CAPACITY UNITS- CONVERT TO TONS / YEAR
none of the above
none of the above
none of the above
none of the above
none of the above


In [37]:
print (composters_new)

          SWISNo                                 Site Name  \
184   28-AA-0026             Upper Valley Disposal Service   
194   30-AB-0378     Golden Rain Foundation Composting Op.   
329   43-AN-0017             Newby Island Compost Facility   
365   50-AA-0016  Gilton Resource Recovery Composting Fac.   
408   24-AA-0029          Billy Wright Composting Facility   
...          ...                                       ...   
6140  39-AA-0058                            Advanced Soils   
6142  49-AA-0425            SCWS Wood Processing Operation   
6144  30-AB-0469                       Bee Canyon Greenery   
6158  47-AA-0059   UC Davis Cooperation Extension Siskiyou   
6159  33-AA-0374                        MDE Compost Site 1   

     Site Operational Status Site Regulatory Status  Latitude  Longitude  \
184                   Active              Permitted  38.47512 -122.43835   
194                   Active           Notification  33.60377 -117.72490   
329                   Activ

In [38]:
#converting to a shapefile
## Use geopandas to make spatial
# from: https://geohackweek.github.io/vector/04-geopandas-intro/
geometry = [Point(xy) for xy in zip(composters_new['Longitude'], composters_new['Latitude'])]
gdf = GeoDataFrame(composters_new, geometry=geometry)

In [39]:
# check length to make sure it matches df
len(geometry)

# In[14]:

# gdf.plot(marker='*', color='green', markersize=50, figsize=(3, 3))

gdf

,SWISNo,Site Name,Site Operational Status,Site Regulatory Status,Latitude,Longitude,Site ZIP Code_x,County_x,County_x,ARB District_x,...,Throughput,ThroughputUnits,RemainingCapacity,Capacity,CapacityUnits,Total Acreage,DisposalAcreage,Waste Type,cap_tpy,geometry
184,28-AA-0026,Upper Valley Disposal Service,Active,Permitted,38.47512,-122.43835,94574,Napa,Napa,Bay Area,...,34000.0,Tons per year,NaN,34000.0,Tons per year,20.0,NaN,Agricultural,34000.000000,POINT (-122.43835 38.47512)
194,30-AB-0378,Golden Rain Foundation Composting Op.,Active,Notification,33.60377,-117.72490,92654,Orange,Orange,South Coast,...,20.0,Tons per day,NaN,10000.0,Cubic Yards per year,3.0,NaN,Green Materials,4464.285714,POINT (-117.72490 33.60377)
329,43-AN-0017,Newby Island Compost Facility,Active,Permitted,37.45470,-121.92300,95035,Santa Clara,Santa Clara,Bay Area,...,700.0,Tons per day,NaN,146600.0,Cubic Yards per year,18.0,NaN,Agricultural,65446.428571,POINT (-121.92300 37.45470)
365,50-AA-0016,Gilton Resource Recovery Composting Fac.,Active,Permitted,37.62099,-120.92583,95357,Stanislaus,Stanislaus,San Joaquin Valley Unified,...,471.0,Tons per day,NaN,471.0,Tons per day,19.0,NaN,Agricultural,171915.000000,POINT (-120.92583 37.62099)
408,24-AA-0029,Billy Wright Composting Facility,Active,Notification,37.04268,-120.96817,93522,Merced,Merced,San Joaquin Valley Unified,...,300.0,Cubic Yards per Day,NaN,9999.0,Cubic Yards per month,13.2,NaN,Agricultural,53566.071429,POINT (-120.96817 37.04268)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6140,39-AA-0058,Advanced Soils,Active,Permitted,37.83379,-120.98238,95320,San Joaquin,San Joaquin,San Joaquin Valley Unified,...,3125.0,Tons per month,NaN,37500.0,Cubic Yards per year,NaN,NaN,Agricultural,16741.071429,POINT (-120.98238 37.83379)
6142,49-AA-0425,SCWS Wood Processing Operation,Active,Notification,38.52434,-122.79242,95492,Sonoma,Sonoma,Bay Area,...,199.0,Tons per day,NaN,72996.0,Tons per year,3.5,NaN,Green Materials,72996.000000,POINT (-122.79242 38.52434)
6144,30-AB-0469,Bee Canyon Greenery,Active,Permitted,33.71730,-117.69666,92618,Orange,Orange,South Coast,...,437.0,Tons per day,NaN,130000.0,Cubic Yards per year,30.0,NaN,Agricultural,58035.714286,POINT (-117.69666 33.71730)
6158,47-AA-0059,UC Davis Cooperation Extension Siskiyou,Active,Notification,41.57324,-121.28222,96134,Siskiyou,Siskiyou,Modoc,...,NaN,NaN,NaN,70.0,Cubic Yards per Day,1.0,NaN,Dead Animals,11406.250000,POINT (-121.28222 41.57324)


In [40]:
# In[15]:

composters_new.head()



,SWISNo,Site Name,Site Operational Status,Site Regulatory Status,Latitude,Longitude,Site ZIP Code_x,County_x,County_x,ARB District_x,...,Throughput,ThroughputUnits,RemainingCapacity,Capacity,CapacityUnits,Total Acreage,DisposalAcreage,Waste Type,cap_tpy,geometry
184,28-AA-0026,Upper Valley Disposal Service,Active,Permitted,38.47512,-122.43835,94574,Napa,Napa,Bay Area,...,34000.0,Tons per year,NaN,34000.0,Tons per year,20.0,NaN,Agricultural,34000.000000,POINT (-122.43835 38.47512)
194,30-AB-0378,Golden Rain Foundation Composting Op.,Active,Notification,33.60377,-117.72490,92654,Orange,Orange,South Coast,...,20.0,Tons per day,NaN,10000.0,Cubic Yards per year,3.0,NaN,Green Materials,4464.285714,POINT (-117.72490 33.60377)
329,43-AN-0017,Newby Island Compost Facility,Active,Permitted,37.45470,-121.92300,95035,Santa Clara,Santa Clara,Bay Area,...,700.0,Tons per day,NaN,146600.0,Cubic Yards per year,18.0,NaN,Agricultural,65446.428571,POINT (-121.92300 37.45470)
365,50-AA-0016,Gilton Resource Recovery Composting Fac.,Active,Permitted,37.62099,-120.92583,95357,Stanislaus,Stanislaus,San Joaquin Valley Unified,...,471.0,Tons per day,NaN,471.0,Tons per day,19.0,NaN,Agricultural,171915.000000,POINT (-120.92583 37.62099)
408,24-AA-0029,Billy Wright Composting Facility,Active,Notification,37.04268,-120.96817,93522,Merced,Merced,San Joaquin Valley Unified,...,300.0,Cubic Yards per Day,NaN,9999.0,Cubic Yards per month,13.2,NaN,Agricultural,53566.071429,POINT (-120.96817 37.04268)


In [41]:
gdf = gdf.set_crs('epsg:4326') #{'init' :'epsg:4326'}

gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [45]:
# gdf.head()
print("exporting shapefile")

# OUT_DIR = "../data/swis/clean"
out = opj(OUT_DIR,"clean_swis_sasha.shp")



exporting shapefile


In [47]:

# type(gdf)

gdf.to_file(driver='ESRI Shapefile', filename=out)

print("p SWIS PRE_PROCESSING DONE RUNNING")

# return gdf

p SWIS PRE_PROCESSING DONE RUNNING


/Users/anayahall/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until
